This is a list of function to process nw images and not nw images into train and test dataset

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from IPython.display import display, Image
%matplotlib inline
from numpy import random

In [2]:
# extract function to extract all the subfolder under root
def extract(root):
    """
    Argument: 
    root -- name or dir of a folder
    Return:
    data_folder -- subfolders of root
    """
    data_folders = [os.path.join(root, d) for d in sorted(os.listdir(root))]
    print data_folders
    return data_folders

In [ ]:
def file_split(filenames, train_size):
    """
    Argument:
    filenames -- list of filenames
    train_size  -- ratio of train dataset (e.g. 0.8)
    Return:
    train_filenames -- list of train filenames
    test_filenames -- list of test filenames
    """
    filenames = sorted(filenames)
    random.seed(230)
    random.shuffle(filenames)
    split_1 = int(train_size * len(filenames))
    train_filenames = filenames[:split_1]
    test_filenames = filenames[split_1:]
    return train_filenames, test_filenames
    

In [3]:
# load function to load imgs from folder (contains several subfolders) and output a dataset and a labels
# import image


def load(data_folders,max_num_images, image_height, image_width, pixel_depth):
    """
    Argument: 
    data_folders -- folders of different classes. Currently is nw and not_nw
    max_num_images -- max number of images can pad into dataset. 
    image_height -- height of image
    image_width -- width of image
    pixel_depth -- depth of image, usually 225.0
    Return:
    Dataset -- dataset contains all the images in the folder, dimesion is [num_images, image_height, image_weith]
    labels -- labels of images, dimension in [num_images], data range in range(classes)
    """
    print data_folders
    dataset = np.ndarray(  # create a ndarray with (max_num_images, n_H, n_W)
        shape=(max_num_images, image_height, image_width), dtype=np.float32) 
    labels = np.ndarray(shape=(max_num_images), dtype=np.int32)
    label_index = 0 # label index start from 0
    image_index = 0 # image_index start from 0
    for folder in data_folders:
        print folder
        print os.listdir(folder)
        for image in os.listdir(folder):
            image_file = os.path.join(folder, image)
            try:
                image_data = (plt.imread(image_file).astype(float) -
                              pixel_depth / 2) / pixel_depth # normalize image
                if image_data.shape != (image_height, image_width):
                    raise Exception('Unexpected image shape: %s' % str(image_data.shape))
                dataset[image_index, :, :] = image_data
                labels[image_index] = label_index
                image_index += 1 # read another image
            except IOError as e:
                print 'Could not read:', image_file, ':', e, '- it\'s ok, skipping.'
        label_index += 1 # next folder, next label
    num_images = image_index
    dataset = dataset[0:num_images, :, :]
    labels = labels[0:num_images]
    print 'Full dataset tensor:', dataset.shape
    print 'Mean:', np.mean(dataset)
    print 'Standard deviation:', np.std(dataset)
    print 'Labels:', labels.shape
    return dataset, labels